# **Análise de dados do SINAN sobre atendimento de violência contra a mulher em unidades de saúde (2017 a 2019)**

    O presente estudo tem como objetivo coletar informações da base de dados do SINAN para traçar um perfil sobre a violência contra as mulheres, abordando as características do ocorrido, das vítimas e de seus agressores.  
    A maioria dos estudos sobre violência contra mulheres se concentram em feminicídio ou então em dados obtidos nas bases da área de segurança pública. Porém muitas mulheres que sofrem esse tipo de violência, por inúmeras razões, decidem não denunciar seu agressor. Já a base do SINAN é alimentada a partir das notificações feitas por unidades de saúde assim que a vítima procura por atendimento e essa notificação é obrigatória. Portanto os dados aqui analisados são de mulheres que precisaram de atendimento após sofrer uma agressão, e não, necessariamente, as que fizeram denúncia à polícia.   

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_dados_violencia = pd.read_csv("/kaggle/input/sinan-violencia-2017-2019/SINAN-VIOL-2017-2019.csv", low_memory=False)
df_dados_violencia.shape

O dataset bruto tem 1.063.056 registros e 161 colunas. Inicialmente exclui os registros das vítimas do sexo masculino e as em que a lesão foi autoprovocada (tentativa de suicídio). Foram escolhidas 31 variáveis e após o processo de limpeza e tratamento dos dados restaram 564.432 registros para serem analisados.

In [ ]:
df_dados_violencia = df_dados_violencia[['level_0', 'level_1','DT_NOTIFIC','DT_OCOR', 'DT_NASC', 'NU_IDADE_N', 'CS_SEXO', 
                                         'CS_RACA', 'CS_ESCOL_N', 'LOCAL_OCOR', 'LES_AUTOP', 'VIOL_FISIC', 'VIOL_PSICO', 
                                         'VIOL_TORT', 'VIOL_SEXU', 'SEX_ASSEDI', 'SEX_ESTUPR', 'SEX_EXPLO', 'SEX_PORNO', 
                                         'SEX_OUTRO', 'REL_PAI', 'REL_MAE', 'REL_PAD', 'REL_MAD', 'REL_CONJ', 'REL_EXCON', 
                                         'REL_NAMO', 'REL_EXNAM', 'REL_FILHO', 'REL_IRMAO', 'REL_CONHEC', 'REL_DESCO',
                                         'AUTOR_SEXO', 'OUT_VEZES']]

df_dados_violencia.rename(columns = {'level_0': 'UF', 'level_1': 'ANO'}, inplace = True)
df_dados_violencia.info()

## **Limpeza e tratamento dos dados**

Esse dataset tem 161 campos, então vamos excluir os campos que não estão relacionados com violência contra a mulher e os que não trazem nenhuma informação que possa ser útil para este estudo.

In [ ]:
df_dados_violencia.dtypes
df_dados_violencia.isna().sum().sort_values()

Como o objetivo do estudo é fazer uma análise sobre a violência contra mulheres, vamos excluir os registros do sexo masculino e ignorados(CS_SEXO = M ou I) , e também os que tratam de lesões auto provocadas (LES_AUTOP = 1)

In [ ]:
df_dados_violencia = df_dados_violencia[df_dados_violencia['CS_SEXO']=='F']
df_dados_violencia = df_dados_violencia[df_dados_violencia['LES_AUTOP']!=1]
df_dados_violencia.drop(['CS_SEXO', 'LES_AUTOP', 'DT_NOTIFIC'], axis = 1, inplace = True)
df_dados_violencia.dropna(subset=['DT_OCOR'], inplace = True)

df_dados_violencia['UF'] = df_dados_violencia['UF'].astype('category')
df_dados_violencia['ANO'] = df_dados_violencia['ANO'].astype('category')
df_dados_violencia['REL_PAI'] = pd.to_numeric(df_dados_violencia['REL_PAI'], errors='coerce')
df_dados_violencia['DT_NASC'] = pd.to_datetime(df_dados_violencia['DT_NASC'])      
df_dados_violencia['DT_OCOR'] = pd.to_datetime(df_dados_violencia['DT_OCOR'])
df_dados_violencia.info()

In [ ]:
#Pandas Profiling fornece de forma rápida e resumida informações sobre o nosso conjunto de dados, conseguimos visualizar facilmente 
#o nome das colunas e seus tipos, a quantidade de linhas e de colunas, e a porcentagem de valores Missing (dados faltantes) 
import pandas_profiling

#pandas_profiling.ProfileReport(dadosViolencia)

In [ ]:
df_dados_violencia.shape

In [ ]:
df_dados_violencia.isna().sum()

Muitos campos que são importantes para uma boa análise estão com um número alto de valores em branco: Idade, Raça, Escolaridade e Local da ocorrência.  

In [ ]:
# Registros sem idade e sem data de nasc.
sidade_snasc = df_dados_violencia[df_dados_violencia['NU_IDADE_N'].isna() & df_dados_violencia['DT_NASC'].isna()]

print(f'Há {len(sidade_snasc)} registros sem idade e sem a data de nascimento.')

In [ ]:
# Excluir os registros sem idade e sem data de nascimento
df_dados_violencia = df_dados_violencia[(df_dados_violencia['NU_IDADE_N'].notnull() | 
                                         df_dados_violencia['DT_NASC'].notnull())]

**- Tratar o campo idade (NU_IDADE_N)**

De acordo com o dicionário de dados do SINAN, a composição da variável obedece o seguinte critério: 

Primeiro digito:
1. Hora
2. Dia
3. Mês
4. Ano

Ex: 3009 – nove meses, 4018 – dezoito anos

Vamos colocar a idade em anos. Caso seja menor de 1 ano, dividimos (idade em meses)/12.
Se for menor que um mês recebe valor zero.  

In [ ]:
# Colocar idade em anos
def calcula_idade(idade, nasc, ocor):
    if idade != idade: # Retorna True caso idade = nan
        return ocor.year - nasc.year - ((ocor.month, ocor.day) < (nasc.month, nasc.day))
    elif (idade > 4000):
        return idade - 4000
    elif (idade > 3000):
        return (idade - 3000)/12
    elif (idade > 1000):
        return 0
    else:
        return idade
      
df_dados_violencia['NU_IDADE_N'] = df_dados_violencia.apply(lambda row: calcula_idade(row['NU_IDADE_N'], 
                                                                                      row['DT_NASC'], 
                                                                                      row['DT_OCOR']), axis=1)

df_dados_violencia['NU_IDADE_N'].isnull().sum()

In [ ]:
# Criar campo GRUPO_IDADE para agrupar idades e ter a visualização dos dados por faixa etária:
bins= [0, 2, 6, 10, 14, 18, 22, 26, 30, 34, 38, 42, 46, 50, 55, 60, 66, 120]
labels = ['< de 2 anos','2 a 5 anos','6 a 9 anos','10 a 13 anos','14 a 17 anos','18 a 21 anos','22 a 25 anos',
          '26 a 29 anos', '30 a 33 anos','34 a 37 anos','38 a 41 anos','42 a 45 anos','46 a 49 anos','50 a 54 anos',
          '55 a 59 anos','60 a 65 anos','> de 65 anos']

df_dados_violencia['GRUPO_IDADE'] = pd.cut(df_dados_violencia['NU_IDADE_N'], bins=bins, labels=labels, 
                                           right=False).astype("category")
df_dados_violencia

In [ ]:
df_dados_violencia['CS_RACA'] = df_dados_violencia['CS_RACA'].astype('category')

mapping = {1 : 'Branca', 
           2 : 'Preta', 
           3 : 'Amarela', 
           4 : 'Parda', 
           5 : 'Indígena', 
           9 : 'Ignorado'}

df_dados_violencia['CS_RACA'] = df_dados_violencia['CS_RACA'].map(mapping)
df_dados_violencia['CS_RACA'].value_counts()

In [ ]:
df_dados_violencia['CS_ESCOL_N'] = df_dados_violencia['CS_ESCOL_N'].astype('category')

mapping = {0 : 'Não alfabetizada', 
           1 : '1ª a 4ª série incompleta do EF', 
           2 : '4ª série completa do EF', 
           3 : '5ª à 8ª série incompleta do EF', 
           4 : 'Ensino fundamental completo', 
           5 : 'Ensino médio incompleto', 
           6 : 'Ensino médio completo',
           7 : 'Educação superior incompleta', 
           8 : 'Educação superior completa', 
           9 : 'Ignorado ou em branco', 
           10 : 'Não se aplica'}

df_dados_violencia['CS_ESCOL_N'] = df_dados_violencia['CS_ESCOL_N'].map(mapping)
df_dados_violencia['CS_ESCOL_N'].value_counts()

In [ ]:
df_dados_violencia['CS_ESCOL_N'].isna().sum()

In [ ]:
df_dados_violencia['LOCAL_OCOR'] = df_dados_violencia['LOCAL_OCOR'].astype('category')

mapping = {1 : 'Residência', 
           2 : 'Habitação coletiva', 
           3 : 'Escola', 
           4 : 'Local de prática esportiva', 
           5 : 'Bar ou similar', 
           6 : 'Via publica',
           7 : 'Comércio/Serviços', 
           8 : 'Industrias/ construção', 
           9 : 'Outro'}

df_dados_violencia['LOCAL_OCOR'] = df_dados_violencia['LOCAL_OCOR'].map(mapping)
df_dados_violencia['LOCAL_OCOR'].value_counts()

In [ ]:
print('Total de registros: '+ str(df_dados_violencia.shape[0]))

# **Perguntas a serem respondidas**

Total por ano e por estado? </br>
Estados com maior e menor número de casos? </br>
Local em que mais ocorre? </br>
Principais tipos de violência? </br>
Sobre os autores, quem mais pratica violência? </br>

**Sobre as vítimas** </br>
Qual sua faixa etária ? </br>
Escolaridade? </br>
Quais raças tiveram mais ocorrência? </br>

In [ ]:
# Inicializa a matplotlib figure
fig,axs = plt.subplots(figsize=(10, 7))

sns.countplot(x=df_dados_violencia['ANO'],label="total")
plt.title('Total de casos por Ano', fontsize=18, fontweight='bold')
axs.set_xlabel(' ', fontsize=12, fontweight='bold')
axs.set_ylabel(' ', fontsize=12, fontweight='bold')

df = df_dados_violencia.groupby("ANO").size()
for i,v in enumerate(df.values) :
  plt.annotate( f"{v:.2f}",(i, v) )

plt.show()

**Nota-se um aumento de notificações em relação ao ano anterior**

Mas aumentou quanto?

In [ ]:
porcent = pd.DataFrame(df_dados_violencia.groupby('ANO').size(), columns=['TOTAL'])
porcent = porcent.pct_change()
porcent

**Em 2018 houve um aumento de 97,8%, em relação ao ano anterior, e de 4,2% em 2019**

# **Número de notificações em cada estado**

In [ ]:
df = pd.Series.to_frame(df_dados_violencia["UF"].value_counts(), name="Total")
df

**São Paulo foi o Estado com o maior número de casos notificados no período de 2017 a 2019, seguido por Minas Gerais e Rio de Janeiro.**

In [ ]:
# Inicializa a matplotlib figure
fig,axs = plt.subplots(figsize=(15, 7))

sns.countplot(x=df_dados_violencia['UF'],label="total", order=df_dados_violencia['UF'].value_counts().index)
plt.title('Total de Casos por Estado', fontsize=18, fontweight='bold')
axs.set_xlabel(' ', fontsize=12, fontweight='bold')
axs.set_ylabel('Total 2017 - 2019', fontsize=12, fontweight='bold')
plt.show()

In [ ]:
# Inicializa a matplotlib figure
fig,axs = plt.subplots(figsize=(15, 7))

sns.countplot(x=df_dados_violencia['UF'], hue=df_dados_violencia['ANO'], order=df_dados_violencia['UF'].value_counts().index)
plt.title('Total de Casos por Estado a Cada Ano', fontsize=18, fontweight='bold')
axs.set_xlabel(' ', fontsize=12, fontweight='bold')
axs.set_ylabel(' ', fontsize=12, fontweight='bold')
axs.legend(title="Ano", loc='upper right') 
plt.show()

**Neste gráfico podemos observar uma tendência de aumento no número de notificações na maioria dos Estados.**

# **Casos por idade**

In [ ]:
print(min(df_dados_violencia['NU_IDADE_N']))
print(max(df_dados_violencia['NU_IDADE_N']))

In [ ]:
plt.subplots(figsize=(10, 7))
df = df_dados_violencia.groupby('GRUPO_IDADE').size()
x = df.values
y = df.keys()
porcent = 100*x/x.sum()
labels = ['{0} - {1:1.1f} %'.format(i,j) for i,j in zip(y, porcent)]
sns.barplot(x=x,y=labels,label="total")
plt.title('Total de casos por idade', fontsize=18, fontweight='bold')
axs.set_xlabel(' ', fontsize=8)
axs.set_ylabel(' ', fontsize=8)
plt.show()

In [ ]:
plt.subplots(figsize=(20, 7))
axs = sns.histplot(df_dados_violencia['NU_IDADE_N'], binrange=[0, 100], fill=True)#kde=True
plt.title('Total de casos por Idade', fontsize=18, fontweight='bold')
axs.set_xlabel('Anos', fontsize=12, fontweight='bold')
axs.set_ylabel(' ', fontsize=12, fontweight='bold')
plt.show()

In [ ]:
print('Moda: ', df_dados_violencia.NU_IDADE_N.mode())

print('Média: ', df_dados_violencia.NU_IDADE_N.mean())

**Vamos agrupar as idades por ciclos de vida:**  

- Criança: 0 a 9 anos  
- Adolescente: 10 a 19 anos  
- Adulto: 20 a 59 anos  
- Idoso: 60 anos e mais  
(o critério aqui é do Ministério da Saúde que adota a faixa etária definida pela OMS)</br>

In [ ]:
# Violencias por ciclo de vida
df = pd.DataFrame(df_dados_violencia['NU_IDADE_N'])

bins = [0, 10, 20, 60, 120]
labels = ['Criança','Adolescente','Adulto','Idoso']
df['Grupos'] = pd.cut(df['NU_IDADE_N'], bins=bins, labels=labels, right=False)

plt.subplots(figsize=(10, 7))
df = df.groupby("Grupos").size()
x = df.values
y = df.keys()
porcent = 100.*x/x.sum()
labels = ['{0} - {1:1.1f} %'.format(i,j) for i,j in zip(y, porcent)]
sns.barplot(x=x,y=labels,label="total")
plt.title('Total de casos por ciclo de vida', fontsize=18, fontweight='bold')
axs.set_xlabel(' ', fontsize=8)
axs.set_ylabel(' ', fontsize=8)
plt.show()

# **Tipos de Violência mais notificados**

Os tipos de violência podem ser:  
- Violência sexual  
- Tortura    
- Violência Fisica  
- Violência Psicologica 

E recebem os seguintes valores:  
1- Sim  
2- Não  
9- Ignorado  

- Violência sexual é dividida em sub-categorias:  
Assédio sexual, estupro, pornografia infantil, exploração sexual ou outras

In [ ]:
def calcula_total_tipo_viol(coluna):
    return len(df_dados_violencia[df_dados_violencia[coluna]==1].dropna().index)

cols = ['VIOL_SEXU',  'VIOL_TORT','VIOL_FISIC', 'VIOL_PSICO']
tp_viol = pd.DataFrame({'total': [calcula_total_tipo_viol(col) for col in cols]},
                       index=['Violência sexual','Tortura','Violência Fisica', 'Violência Psicologica'])

tp_viol = tp_viol.sort_values(by='total', ascending=True)
tp_viol.plot.pie(y='total',figsize=(10, 10),autopct='%.2f', title='Porcentagem de Violência por Tipo', 
                 startangle=90,legend=False, ylabel=None)

**Violência Sexual**

Pode ser:
Assédio sexual, estupro, pornografia infantil, exploração sexual ou outras

In [ ]:
def calcula_total_tipo_viol(coluna):
    return len(df_dados_violencia[df_dados_violencia[coluna]==1].dropna().index)

cols = ['SEX_ESTUPR', 'SEX_ASSEDI', 'SEX_EXPLO', 'SEX_PORNO', 'SEX_OUTRO']
df = pd.DataFrame({'total': [calcula_total_tipo_viol(col) for col in cols]},
                  index=['Estupro','Assédio sexual', 'Exploração sexual', 'Pornografia infantil', 'Outros'])

df = df.sort_values(by='total', ascending=True)
df.plot.pie(y='total',autopct='%.2f', title='Porcentagem de Violencia Sexual', figsize=(10, 10), 
            startangle=90,legend=False, ylabel=None)

df = df.sort_values(by='total', ascending=False)
df

# **Violência de repetição**

Agressão ocorreu outras vezes?

In [ ]:
total = len(df_dados_violencia.index)
df = len(df_dados_violencia[df_dados_violencia['OUT_VEZES'] == 1].index)
porcent = round((df*100/ total), 2)
print(porcent, '% das notificações são de violência de repetição.')

# **Relação das vítimas com o autor da agressão**

A relação da pessoa atendida com o provável autor da agressão

In [ ]:
# Calcular o total de cada tipo de relação com autor(a/es)
def calcula_total_relac(coluna):
    return len(df_dados_violencia[df_dados_violencia[coluna]==1].dropna().index)

cols = ['REL_PAI', 'REL_MAE', 'REL_PAD', 'REL_MAD', 'REL_CONJ', 'REL_EXCON', 'REL_NAMO', 'REL_EXNAM', 'REL_FILHO', 
        'REL_IRMAO', 'REL_CONHEC', 'REL_DESCO']
df = pd.DataFrame({'total': [calcula_total_relac(col) for col in cols]},
                  index=['Pai','Mãe','Padrasto','Madrasta','Conjuge', 'Ex Conjuge', 'Namorado', 'Ex Namorado', 
                         'Filho', 'Irmão', 'Conhecido', 'Desconhecido'])

df = df.sort_values(by='total', ascending=False)
df

In [ ]:
df = df.sort_values(by='total', ascending=True)
df.plot.pie(y='total',figsize=(10, 10),autopct='%.2f', title='Relação com o provável autor', startangle=90,legend=False, ylabel=None)

# **Escolaridade**

- Categoria "Não se aplica" é preenchida automaticamente quando a vítima tem menos de 7 anos.

In [ ]:
df = df_dados_violencia.groupby("CS_ESCOL_N").size()
# incompletos = round((df['Ignorado ou em branco'].sum()/df.sum())*100, 1)
# print('Ignorados ou em branco = ' + str(incompletos)+'%')
      
# df = df.drop(labels=['Ignorado ou em branco', 'Não se aplica'])
df = df.sort_values(ascending=False)
keys = df.keys()
values = df.values

porcent = 100*values/values.sum()
labels = ['{0} - {1:1.1f} %'.format(i,j) for i,j in zip(keys, porcent)]

plt.subplots(figsize=(15, 7))
axs = sns.barplot(x=values, y=labels)
plt.title('Escolaridade', fontsize=16)
axs.set_ylabel(' ', fontsize=12, fontweight='bold')
plt.show()

**Muitos registros sem o preenchimento dessa informação, o que não permite fazer um mapeamento correto da população.**

# **Raça**

Considera-se cor ou raça declarada pela pessoa:  
1-branca  
2- preta  
3- amarela (pessoa que se declarou de raça amarela)  
4- parda (pessoa que se declarou mulata, cabocla, cafuza, mameluca ou mestiça de preto com pessoa de outra cor ou raça)  
5- indígena (pessoa que se declarou indígena ou índia)  

In [ ]:
plt.subplots(figsize=(10, 7))
axs = sns.countplot(x=df_dados_violencia['CS_RACA'],label="total", order=df_dados_violencia['CS_RACA']
                    .value_counts()
                    .index)
plt.title('Total de casos por raça', fontsize=18, fontweight='bold')
axs.set_xlabel(' ', fontsize=12, fontweight='bold')
axs.set_ylabel(' ', fontsize=12, fontweight='bold')
plt.show()

In [ ]:
df = df_dados_violencia.groupby("CS_RACA").size().sort_values()

plt.subplots(figsize=[8,8])
keys = df.keys()
values = df.values

porcent = 100*values/values.sum()
labels = ['{0} - {1:1.1f} %'.format(i,j) for i,j in zip(keys, porcent)]
patches, texts = plt.pie(values, startangle=45)
plt.axis('equal')
plt.title("Porcentagem de casos por raça", fontsize=16)
patches, labels, dummy =  zip(*sorted(zip(patches, labels, values),key=lambda x: x[2], reverse=True))
plt.legend(patches, labels, loc='best', bbox_to_anchor=(-0.1, 1.), fontsize=12)
plt.show()

# **Local de ocorrência**

Onde ocorreu a agressão?  

01- Residência  
02- Habitação coletiva  
03- Escola  
04- Local de prática esportiva  
05- Bar ou similar  
06- Via publica  
07- Comércio/Serviços  
08- Industrias/ construção  
09- Outro  
99-Ignorado  

In [ ]:
df = df_dados_violencia.groupby("LOCAL_OCOR").size().sort_values()

plt.subplots(figsize=[8,8])
keys = df.keys()
values = df.values

porcent = 100*values/values.sum()
labels = ['{0} - {1:1.1f} %'.format(i,j) for i,j in zip(keys, porcent)]
patches, texts = plt.pie(values, startangle=45)
#plt.pie(x=df, autopct="%.1f%%", labels=y, pctdistance=0.5, startangle=45)
plt.axis('equal')
plt.title("Local de Ocorrência", fontsize=16)
patches, labels, dummy =  zip(*sorted(zip(patches, labels, values),key=lambda x: x[2],reverse=True))
plt.legend(patches, labels, loc='best', bbox_to_anchor=(-0.1, 1.), fontsize=12)
plt.show()

# **Resultados**  


    De 2017 para 2018 houve um crescimento de 9,78% no número de notificações, e de 4,27% em 2019. O maior número de notificações foram no estado de São Paulo com 139.619, seguido por Minas Gerais com 75.810 e Rio de Janeiro com 66.147 casos registrados nos três anos do estudo. 

    A violência física é a que mais foi notificada, tendo como principal agressor o companheiro da vítima. A situação geralmente ocorre dentro da própria residência e quase 40% das notifcações foram de violência de repetição.  
    
    Quanto às vítimas, a maioria está na faixa etária de 14 a 25 anos, pardas e com ensino fundamental incompleto. Isso não significa necessariamente que essas sejam as categorias com maiores fatores de risco já que como nossa fonte de dados é da área da saúde ela se limita as mulheres que possuem acesso ao SUS.

# **Conclusão**

    O presente estudo tinha como objetivo fazer uma análise de dados, traçando um perfil da violência contra a mulher no Brasil a partir dos dados notificados, principalmente, pelas unidades de saúde. Infelizmente muito dos dados encontrados na base do SINAN estavam incompletos, mesmo os campos de preenchimento obrigatório, ou continham erros. Outro problema encontrado foi que são muitas variáveis, 161, e nenhum dicionário de dados encontrado correnpondiam essas mesmas variáveis. Algumas estavam no dataset mas não havia registro dela no dicionário de dados, outras estavam descritas no dicionário porém não estavam presentes no dataset. Isso acaba impedindo que seja feita uma análise e mapeamento mais adequado e que represente a população analisada.